In [0]:
from pyspark.sql.functions import *

# Read cleaned data from Silver
silver_df = spark.table("silver_nyc_taxi")

# Daily location aggregates
daily_location = silver_df.groupBy(
    to_date("pickup_datetime").alias("report_date"),
    "pickup_location_id"
).agg(
    count("*").alias("total_trips"),
    avg("fare_amount").alias("avg_fare_amount"),
    avg("trip_duration_min").alias("avg_trip_duration")
)

# Write to Gold with merge
daily_location.createOrReplaceTempView("updates")

spark.sql("""
  MERGE INTO gold_daily_location target
  USING updates source
  ON source.report_date = target.report_date
    AND source.pickup_location_id = target.pickup_location_id
  WHEN MATCHED THEN UPDATE SET *
  WHEN NOT MATCHED THEN INSERT *
""")